In [1]:
import trimesh.transformations as tt

import numpy as np

In [52]:
def rotate_to_world_origin(camera_origin):
    # construct a 3x3 rotation matrix to a coordinate frame where:
    # Z axis points to world origin aka center of a mesh
    # Y axis points down
    # X axis is computed as Y cross Z
    
    camera_origin = np.asanyarray(camera_origin)

    e_z = -camera_origin / np.linalg.norm(camera_origin)  # Z axis points to world origin aka center of a mesh
    e_y = np.array([0, 0, -1])  # proxy to Y axis pointing directly down
                                # note that real e_y must be 
                                # 1) orthogonal to e_z;
                                # 2) lie in the plane spanned by e_y and e_z;
                                # 3) point downwards so <e_y, [0, 0, -1]> >= <e_y, [0, 0, +1]>
                                # 4) unit norm
    gamma = np.dot(e_y, e_z)
    e_y = -gamma / (1 + gamma**2) * e_z + 1. / (1 + gamma**2) * e_y
    if np.dot(e_y, [0, 0, -1]) < np.dot(e_y, [0, 0, 1]):
        e_y *= -1
    e_y /= np.linalg.norm(e_y)
    e_x = np.cross(e_y, e_z)  # X axis 
    R = np.array([e_x, e_y, e_z])
    return R

In [155]:
class CameraPose:
    def __init__(self, transform):
        self._camera_to_world_4x4 = transform
        # always store transform from world to camera frame
        self._world_to_camera_4x4 = np.linalg.inv(self._camera_to_world_4x4)  

    @classmethod
    def from_camera_to_world(cls, rotation=None, translation=None):
        """Create camera pose from camera to world transform.
        
        :param rotation: 3x3 rotation matrix of camera frame axes in world frame
        :param translation: 3d location of camera frame origin in world frame
        """
        rotation = np.identity(3) if None is rotation else np.asanyarray(rotation)
        translation = np.zeros(3) if None is translation else np.asanyarray(translation)

        transform = np.identity(4)
        transform[:3, :3] = rotation
        transform[:3, 3] = translation

        return cls(transform)
    
    @classmethod
    def from_camera_axes(cls, R=None, t=None):
        """Compute 4x4 camera pose from camera axes given in world frame.
        
        :param R: a list of 3D basis vectors (cx, cy, cz) defined in world frame
        :param translation: 3D vector defining location of camera origin in world frame
        """
        if None is R:
            R = np.identity(3)
        
        return cls.from_camera_to_world(rotation=R.T, translation=t)
    
    def world_to_camera(self, points):
        """Transform points from world to camera coordinates.
        Useful for understanding where the objects are, as seen by the camera.
        
        :param points: either n * 3 array, or a single 3-vector
        """
        points = np.atleast_2d(points)
        return tt.transform_points(points, self._world_to_camera_4x4)
    
    def camera_to_world(self, points, translate=True):
        """Transform points from camera to world coordinates.
        Useful for understanding where objects bound to camera 
        (e.g., image pixels) are in the world.
        
        :param points: either n * 3 array, or a single 3-vector
        """
        points = np.atleast_2d(points)
        return tt.transform_points(points, self._camera_to_world_4x4, translate=translate)
    
    @property
    def world_to_camera_4x4(self):
        return self._world_to_camera_4x4

    @property
    def camera_to_world_4x4(self):
        return self._camera_to_world_4x4
    
    @property
    def frame_origin(self):
        """Return camera frame origin in world coordinates."""
        return pose.camera_to_world_4x4[:3, 3]
    
    @property
    def frame_axes(self):
        """Return camera axes: a list of 3D basis 
        vectors (cx, cy, cz) defined in world frame"""
        return pose.camera_to_world_4x4[:3, :3].T

    def compose_world_to_camera(self, other_pose):
        """Compose camera poses C_1, C_2, ... (defined relative to each other), 
        computing transforms from world frame to an innermost camera frame.
        
        Equivalent to: 
        x_world = <some point>
        other_pose.world_to_camera(
            pose.world_to_camera(
                x_world
            )
        )
        """
        composed_world_to_camera_4x4 = np.dot(other_pose.world_to_camera_4x4, self._world_to_camera_4x4)
        composed_camera_to_world_4x4 = np.linalg.inv(composed_world_to_camera_4x4)
        return CameraPose(composed_camera_to_world_4x4)
    
    def compose_camera_to_world(self, other_pose):
        """Compose camera poses C_1, C_2, ... (defined relative to each other), 
        computing transforms from innermost camera frame to the world frame.
        
        Equivalent to: 
        x_local = <some point>
        pose.camera_to_world(
            pose_local.camera_to_world(
                x_local
            )
        )
        """
        composed_camera_to_world_4x4 = np.dot(self._camera_to_world_4x4, other_pose.camera_to_world_4x4, )
        return CameraPose(composed_camera_to_world_4x4)

In [136]:
t = np.array([1, 1, 0])
R = rotate_to_world_origin(t)
# R = np.identity(3)

In [146]:
pose = CameraPose.from_camera_axes(R=R, t=t)

In [147]:
pose_local = CameraPose.from_camera_axes(R=np.identity(3), t=np.array([1, 1, 0]))

In [148]:
pose_local.world_to_camera_4x4, pose.world_to_camera_4x4

(array([[ 1.,  0.,  0., -1.],
        [ 0.,  1.,  0., -1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]),
 array([[-0.70710678,  0.70710678, -0.        , -0.        ],
        [-0.        , -0.        , -1.        , -0.        ],
        [-0.70710678, -0.70710678, -0.        ,  1.41421356],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [140]:
composed = pose.compose_right(pose_local)

In [141]:
composed.world_to_camera_4x4

array([[-0.70710678,  0.70710678, -0.        , -1.        ],
       [-0.        , -0.        , -1.        , -1.        ],
       [-0.70710678, -0.70710678, -0.        ,  1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [143]:
np.dot(pose_local.world_to_camera_4x4, pose.world_to_camera_4x4),

(array([[-0.70710678,  0.70710678,  0.        , -1.        ],
        [ 0.        ,  0.        , -1.        , -1.        ],
        [-0.70710678, -0.70710678,  0.        ,  1.41421356],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),)

In [94]:
np.dot(
    np.dot(pose_local.world_to_camera_4x4, pose.world_to_camera_4x4),
    np.array([1., 1., 1., 1.]).T
)

array([-1., -2.,  0.,  1.])

In [90]:
pose.camera_to_world_4x4

array([[-0.70710678,  0.        , -0.70710678,  1.        ],
       [ 0.70710678,  0.        , -0.70710678,  1.        ],
       [ 0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [70]:
x_world = np.array([1., 1., 1.])

In [92]:
pose.world_to_camera(x_world)

array([[ 0., -1.,  0.]])

In [91]:
pose_local.world_to_camera(
    pose.world_to_camera(x_world)
)

array([[-1., -2.,  0.]])

In [95]:
x_local = np.array([0., 0., 0.])

In [96]:
pose_local.camera_to_world(
    x_local
)

array([[1., 1., 0.]])

In [97]:
pose.camera_to_world(
    pose_local.camera_to_world(
        x_local
    )
)

array([[ 0.29289322,  1.70710678, -1.        ]])

In [149]:
np.dot(pose.camera_to_world_4x4, pose_local.camera_to_world_4x4)

array([[-0.70710678,  0.        , -0.70710678,  0.29289322],
       [ 0.70710678,  0.        , -0.70710678,  1.70710678],
       [ 0.        , -1.        ,  0.        , -1.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [151]:
pose.compose_camera_to_world(pose_local).camera_to_world_4x4

array([[-0.70710678,  0.        , -0.70710678,  0.29289322],
       [ 0.70710678,  0.        , -0.70710678,  1.70710678],
       [ 0.        , -1.        ,  0.        , -1.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [153]:
pose.compose_camera_to_world(pose_local).world_to_camera_4x4

array([[-0.70710678,  0.70710678, -0.        , -1.        ],
       [-0.        , -0.        , -1.        , -1.        ],
       [-0.70710678, -0.70710678, -0.        ,  1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [154]:
np.dot(pose_local.world_to_camera_4x4, pose.world_to_camera_4x4),

(array([[-0.70710678,  0.70710678,  0.        , -1.        ],
        [ 0.        ,  0.        , -1.        , -1.        ],
        [-0.70710678, -0.70710678,  0.        ,  1.41421356],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),)